In [1]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [2]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.7.1+cu126 with CUDA 1208 (you have 2.6.0+cu124)
    Python  3.9.23 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.11: Fast Llama patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)

Unsloth 2025.7.11 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

## Instruction:
Company database: {}

### Input:
SQL Prompt: {}

### Response:
SQL: {}

Explanation: {}"""

In [5]:
EOS_TOKEN = tokenizer.eos_token

In [6]:
def formatting_prompts_func(examples):
    company_databases = examples["sql_context"]
    prompts           = examples["sql_prompt"]
    sqls              = examples["sql"]
    explanations      = examples["sql_explanation"]
    texts = []
    for db, inp, sql, expl in zip(company_databases, prompts, sqls, explanations):
        txt = alpaca_prompt.format(db, inp, sql, expl)
        texts.append(txt + EOS_TOKEN)
    return {"text": texts}

In [7]:
from datasets import load_dataset

train_dataset = load_dataset("gretelai/synthetic_text_to_sql", split="train")
test_dataset = load_dataset("gretelai/synthetic_text_to_sql", split="test")

train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
test_dataset = test_dataset.map(formatting_prompts_func, batched=True)

In [8]:
print(train_dataset['text'][0])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

## Instruction:
Company database: CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');

### Input:
SQL Prompt: What is the total volume of timber sold by each salesperson, sorted by salesperson?

### Response:
SQL: SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id GROUP BY salesperson_id, name ORDER BY total_volume DESC;

Explanation: Joins timber_sales

In [9]:
print(test_dataset['text'][0])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

## Instruction:
Company database: CREATE TABLE creative_ai (application_id INT, name TEXT, region TEXT, explainability_score FLOAT); INSERT INTO creative_ai (application_id, name, region, explainability_score) VALUES (1, 'ApplicationX', 'Europe', 0.87), (2, 'ApplicationY', 'North America', 0.91), (3, 'ApplicationZ', 'Europe', 0.84), (4, 'ApplicationAA', 'North America', 0.93), (5, 'ApplicationAB', 'Europe', 0.89);

### Input:
SQL Prompt: What is the average explainability score of creative AI applications in 'Europe' and 'North America' in the 'creative_ai' table?

### Response:
SQL: SELECT AVG(explainability_score) FROM creative_ai WHERE region IN ('Europe', 'North America');

Explanation: This SQL query calculates the average explainability score of creative AI applications in 'Europe' and 'North America' in the 'creative_ai' t

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback


training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    max_steps=500,
    warmup_steps=50,
    learning_rate=2e-4,
    fp16=torch.cuda.is_available() and not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_steps=10,
    output_dir="outputs_sql",
    report_to=[],
    seed=3407,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    packing=False,
    args=training_args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/5851 [00:00<?, ? examples/s]

In [ ]:
if __name__ == "__main__":
    trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
50,0.501100,0.517205
100,0.500700,0.490448
150,0.486900,0.481264
200,0.470400,0.473014
250,0.450900,0.467308
300,0.481100,0.462263


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [2]:
results = trainer.evaluate()
print(f"Evaluation Results: {results}")

NameError: name 'trainer' is not defined

In [3]:
output_dir = "/content/outputs_sql"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model & tokenizer saved locally to {output_dir}")

NameError: name 'model' is not defined

In [9]:
!ls -al /content/outputs_sql

ls: cannot access '/content/outputs_sql': No such file or directory


In [5]:
!zip -r -q /content/outputs_sql.zip /content/outputs_sql


zip error: Nothing to do! (try: zip -r -q /content/outputs_sql.zip . -i /content/outputs_sql)


In [7]:
from google.colab import files

files.download("/content/outputs_sql.zip")

FileNotFoundError: Cannot find file: /content/outputs_sql.zip